# Parsing results notebook

Selecting devices...

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Imports...

In [ ]:
import sys
sys.path.insert(1, "..")
from datasets.ssl_dataset import SSL_Dataset
from datasets.data_utils import get_data_loader
import matplotlib.pyplot as plt
import torch
from glob import glob
from utils import net_builder
from train_utils import mcc
import pandas as pd
from tqdm import tqdm

Loading dataset...

In [ ]:
dataset_dir="/home/gabrielemeoni/project/END2END/MSMatch/data/"
test_dset = SSL_Dataset(name="thraws_swir_test", train=False, data_dir=dataset_dir)

test_dset_basic = test_dset.get_dset()
num_classes = test_dset.num_classes
num_channels = test_dset.num_channels
eval_loader = get_data_loader(test_dset_basic, 8, num_workers=1)
    

Checkpoint path...

In [ ]:
checkpoint_path="/home/gabrielemeoni/project/END2END/MSMatch/checkpoints/paper_train"

Listing tests....

In [ ]:
test_checkpoint_seed_paths=sorted(glob(os.path.join(checkpoint_path, "*")))

In [ ]:
seeds=[]
test_results_mcc_seed=[]
test_results_acc_seed=[]

for test_seed_path in tqdm(test_checkpoint_seed_paths, desc="Processing seed..."):
    seeds.append(int(test_seed_path.split(os.sep)[-1].split("_")[-1]))
    upsampling_values_test_paths=sorted(glob(os.path.join(test_seed_path, "*")))
    test_results_acc_seed_upsample=[]
    test_results_mcc_seed_upsample=[]
    upsampling_values=[]
    for test in upsampling_values_test_paths:
        upsampling_values.append(int(test.split("_{")[1].split("}")[0]))

        #Exploring the whole path until you reach the final directory
        while(len(glob(os.path.join(test, "*"))) == 1):
            test=os.path.join(test, glob(os.path.join(test, "*"))[0])
            
        checkpoint_path = os.path.join(test, "model_best.pth")
        checkpoint = torch.load(checkpoint_path)
        load_model = (checkpoint["eval_model"])

        _net_builder = net_builder(
            "efficientnet-lite0",
            False,
            {
                "depth": 28,
                "widen_factor": 2,
                "leaky_slope": 0.1,
                "dropRate": 0.0,
            },
        )
        net = _net_builder(num_classes=num_classes, in_channels=num_channels)
        net.load_state_dict(load_model)
        if torch.cuda.is_available():
            net.cuda()
        net.eval()
        
        acc = 0.0
        y_true=[]
        y_pred=[]
        n=0
        with torch.no_grad():
            for image, target in eval_loader:
                image = image.type(torch.FloatTensor).cuda()
                logit = net(image)
                y_pred+=list(logit.cpu().max(1)[1])
                y_true+=list(target)
                acc += logit.cpu().max(1)[1].eq(target).sum().numpy()

                if n == 0:
                        pred=logit
                        correct=target
                        n+=1
                else:
                    pred=torch.cat((pred, logit), axis=0)
                    correct=torch.cat((correct, target), axis=0)
        test_results_acc_seed_upsample.append(acc / len(test_dset_basic))
        test_results_mcc_seed_upsample.append(mcc(pred, correct))
    
    upsampling_values=["up_" + str(upsampling_value) for upsampling_value in upsampling_values]
    test_results_acc_seed.append(dict(zip(upsampling_values, test_results_acc_seed_upsample)))
    test_results_mcc_seed.append(dict(zip(upsampling_values, test_results_mcc_seed_upsample)))
    
seeds_sorted_idx=sorted(range(len(seeds)),key=seeds.__getitem__)
seeds_sorted=["seed_"+str(seeds[n]) for n in seeds_sorted_idx]
test_results_acc_seed_sorted=[test_results_acc_seed[n] for n in seeds_sorted_idx]
test_results_mcc_seed_sorted=[test_results_mcc_seed[n] for n in seeds_sorted_idx]
    
tests_results_acc_dict=dict(zip(seeds_sorted, test_results_acc_seed_sorted))
tests_results_mcc_dict=dict(zip(seeds_sorted, test_results_mcc_seed_sorted))



Accuracy results.

In [ ]:
acc_df=pd.DataFrame.from_dict(tests_results_acc_dict, orient='index',columns=upsampling_values)
acc_df

Accuracy mean over seeds.

In [ ]:
acc_df.mean(axis=0)

MCC results.

In [ ]:
mcc_df=pd.DataFrame.from_dict(tests_results_mcc_dict, orient='index',columns=upsampling_values)
mcc_df

MCC mean over seeds.

In [ ]:
mcc_df.mean(axis=0)